# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

#Set some global options
pd.set_option('max_columns', 100) #Its nice to see all columns

#Reproducibilty
np.random.seed(42)

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

----------------

## My work and solutions

So this excercise gives us a dataset analyzing racial discrimination in hiring in the US. As always, before diving into the questions we should take a look at the given dataset. So let's do that now.

In [3]:
data.sample(5)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
1654,19,19,4,4,2,0,1,1,0,267,4,1,1,1,0,Allison,f,w,1.0,0.0,0.0,b,s,51.0,0.022392,0.221549,9.614338,0.356164,0.187566,9.520982,1.0,,,0.0,NaN,NaN,NaN,NaN,NaN,0.00315,0.983229,10.873944,0.054705,0.309197,9.955795,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
1101,b,15,4,4,6,0,1,0,1,195,1,1,1,1,1,Tanisha,f,b,1.0,0.0,0.0,c,a,626.0,0.003419,0.904997,10.310918,0.213802,0.142228,9.557964,1.0,some,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,
1149,295,15,2,3,8,1,0,0,1,263,4,0,0,0,0,Latoya,f,b,0.0,1.0,0.0,b,s,137.0,0.009224,0.965915,10.144078,0.177731,0.145395,9.593764,0.0,5,,1.0,22.500000,147.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private
2763,b,3,2,2,6,0,0,0,0,189,1,0,0,1,0,Latoya,f,b,0.0,1.0,0.0,c,a,941.0,0.954597,0.042804,10.276773,0.197444,0.109380,9.304468,0.0,,,0.0,46.400002,362.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private
3145,103,33,4,4,14,0,1,0,0,13,1,1,0,1,1,Latoya,f,b,1.0,0.0,0.0,b,s,4.0,0.586862,0.337866,10.291060,0.180473,0.132419,9.400630,1.0,some,,0.0,37.500000,150.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

There are lots of columns! If the info() method is fully accurate, it seems the data is mostly complete as well. The exercise only asks us to look at the **race** and **call** columns. So we can do that. First, let's drop the other columns and create a new dataframe, df, that we will use for the rest of the exercise.

In [5]:
df = data[['race','call']]

We know there are 4870 rows, so let's figure out how many of each race are present. Is it even or unbalanced?

In [6]:
df.race.value_counts()

b    2435
w    2435
Name: race, dtype: int64

Good, both categories have the same size. Next we can figure out how many callbacks were received, per race. 

In [7]:
df.groupby(['race','call']).size().unstack()

call,0.0,1.0
race,,
b,2278,157
w,2200,235


So we can see that the number of callbacks is not the same, even though the initial sizes were the same. So we want to quantify whether this difference is statistical significant. On to the questions!

### Question 1: What test is appropriate for this problem? Does CLT apply?

The data is independent and identically distributed, as defined in the problem statement. Also there is much more data than is the rule of thumb (30). Because of this, CLT does apply. Given that we want to compare two sample proportions (and not means) with known variances the best test is the *z*-test. This will be a two-sided test. 

### Answer 1: CLT does apply in this scenario and a two-sided *z*-test is appropriate. 

### Question 2: What are the null and alternate hypotheses?

### Answer 2: The null hypothesis is that the difference between the rate (or proportion) of callbacks depending on race is zero. The alternate hypothesis then presumes that the difference is non-zero. 

### Question 3: Compute margin of error, confidence interval, and p-value.

Here we want to calculate the relevant statistics to answer the question. Before that, let's remind ourselves what the rates of callbacks are for the two race categories.

In [8]:
df.groupby(['race','call']).size().apply(lambda x: x/2435.).unstack()

call,0.0,1.0
race,,
b,0.935524,0.064476
w,0.903491,0.096509


Now, we first must use both proportions and determine a pooled, or averaged, proportion. From that we can calculate the standard error. Using that and the difference in proportions (which is the goal of the *z*-test), we can calculate the test statistic. We are not given a confidence level (CL), but we can assume 95% as that is pretty standard. We can use this CL to determine the related *z*-score (1.96 in this case) and multiply by the standard error to get the margin of error. Then we can calculate the confidence interval (CI) and p-value from previous calculations. The full calculations and results are given below.

In [9]:
#Calculate the number of callbacks for each race
num_call_w = sum(df[df.race=='w'].call)
num_call_b = sum(df[df.race=='b'].call)

#Calculate the total rows for each race
num_tot_w = len(df[df.race=='w'].call)
num_tot_b = len(df[df.race=='b'].call)

#Calculate the proportion of callback for each race
prop_w = num_call_w / num_tot_w
prop_b = num_call_b / num_tot_b

#Calculate the pooled proportion
prop_pool = (prop_w * num_tot_w + prop_b * num_tot_b) / (num_tot_w + num_tot_b)

#Calculate the difference in proportions
prop_diff = np.abs(prop_w-prop_b)

#Calculate the standard error between proportions
std_error = np.sqrt(prop_pool * (1-prop_pool) * ((1./num_tot_w) + (1./num_tot_b)))

#Calculate the z-score for this exercise
z = prop_diff / std_error

#Assume a confidence level of 0.95
CL = 0.95

#Calculate the margin of error
margin_error = stats.norm.ppf(1-((1-CL)/2)) * std_error

#Calculate the 95% CI from the difference of proportions
ci_lower, ci_upper = stats.norm.interval(CL, loc=prop_diff, scale=std_error)

#Calculate the p-value 
p_value = (1-stats.norm.cdf(z))*2

print("  %-30s | %s" % ('Calculation', 'Value'))
print(60 * '=')
print("  %-30s | %.8f" % ('Callback proportion for W', prop_w))
print("  %-30s | %.8f" % ('Callback proportion for B', prop_b))
print("  %-30s | %.8f" % ('Difference in proportions', prop_diff))
print("  %-30s | %.8f" % ('Pooled proportions', prop_pool))
print("  %-30s | %.8f" % ('Standard Error', std_error))
print("  %-30s | %.8f" % ('z-test statistic', z))
print("* %-30s | %.8f" % ('Margin of Error', margin_error))
print("* %-30s | %.8f to %.8f" % ('%i%% Confidence Interval' % (CL*100), ci_lower, ci_upper))
print("* %-30s | %.8f" % ('z-test p-value', p_value))

  Calculation                    | Value
  Callback proportion for W      | 0.09650924
  Callback proportion for B      | 0.06447639
  Difference in proportions      | 0.03203285
  Pooled proportions             | 0.08049281
  Standard Error                 | 0.00779689
  z-test statistic               | 4.10841215
* Margin of Error                | 0.01528163
* 95% Confidence Interval        | 0.01675122 to 0.04731449
* z-test p-value                 | 0.00003984


Note the table above gives the results of the calculations we needed for this question. The last three rows, marked by an asterisk, are the quantities needed for this question. 

### Answer 3: See table above. 

### Question 4: Write a story describing the statistical significance in the context or the original problem.

### Answer 4: The p-value from the *z*-test is very small. The CI also supports this p-value since it does not include zero (remember its calculated for the difference in proportions). This means the difference between the two proportions is not zero and we can reject the null hypothesis. In a vacuum, this analysis would indicate that indeed there is a bias in who gets a callback for an employment application. 

### Question 5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

### Answer 5: There are other factors that are likely to play in this scenario. We do not know how much name (which is used as a proxy for race, another complication) actually affects employment applications. Many of these other factors are likely present in the given dataset, as they include info on sex, education, and background. Without looking into these other features of the dataset it would be difficult to say that race (or name) is the most important factor in callback success. A better analysis would use regression/machine learning to map and learn the relationships between all the relavant factors.  

-------------------

Et fin.